In [1]:
from PyQt5.QtGui import *
from PyQt5.QtSvg import *
from PyQt5.QtCore import *
from PyQt5.QtWidgets import *
from PyQt5.QtWebKitWidgets import *
from PyQt5.QtPrintSupport import *

import numpy
import sys
import toyplot.svg
import toyplot.units
import xml.etree.ElementTree as xml

In [2]:
canvas = toyplot.Canvas(width="4in")
axes = canvas.axes(label="Foo", xlabel="Bar", ylabel=u"L\u0302")
mark = axes.plot(numpy.linspace(0, 1)**2)
svg = toyplot.svg.render(canvas)

In [3]:
application = QApplication(sys.argv)

Using QSvgRenderer to generate a PNG:

In [4]:
renderer = QSvgRenderer()
renderer.load(QByteArray(xml.tostring(svg)))
image = QImage(renderer.defaultSize(), QImage.Format_ARGB32)
image.fill(QColor("white"))
painter = QPainter(image)
renderer.render(painter)
painter.end()
image.save("test-qt-svg-renderer.png")

True

Using WebKit to generate a PNG:

In [5]:
page = QWebPage()
page.mainFrame().setContent(xml.tostring(toyplot.svg.render(canvas)), "image/svg+xml")
page.setViewportSize(page.mainFrame().contentsSize())
image = QImage(page.viewportSize(), QImage.Format_ARGB32)
painter = QPainter(image)
page.mainFrame().render(painter)
painter.end()
image.save("test-qt-webkit-renderer.png")

True

Using WebKit to generate a PDF:

In [6]:
page_size = QPageSize(QSizeF(
    toyplot.units.convert((canvas._width, "px"), "pt"),
    toyplot.units.convert((canvas._height, "px"), "pt")), QPageSize.Point, matchPolicy=QPageSize.ExactMatch)

In [7]:
printer = QPrinter()
printer.setPageSize(page_size)
printer.setOutputFormat(QPrinter.PdfFormat)
printer.setOutputFileName("test-qt-webkit-renderer.pdf")
page.mainFrame().print_(printer)
